In [ ]:
import sys
sys.path.append("..")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.dataset import HierarchicalDataset
import pandas as pd

In [ ]:
dataset = HierarchicalDataset(cases_dir="./ILCaseAndMortalityInputV1.csv", # okay
                              interventions_dir="ILInterventions.csv", # okay
                              ifr_dir="ILWeightedFatalityInput.csv", # okay
                              serial_interval_dir="../../data/EU/serial_interval.csv",  # okay
                             )

# next: onward to the STAN interface.

In [80]:
stan_data = dataset.get_stan_data(99)

First non-zero cases is on day 52, and 30 days before 5 days is day 36
84017043 has 81 of data
First non-zero cases is on day 72, and 30 days before 5 days is day 57
84017049 has 60 of data
First non-zero cases is on day 66, and 30 days before 5 days is day 61
84017051 has 56 of data
First non-zero cases is on day 69, and 30 days before 5 days is day 55
84017053 has 62 of data
First non-zero cases is on day 62, and 30 days before 5 days is day 86
84017063 has 31 of data
First non-zero cases is on day 65, and 30 days before 5 days is day 80
84017075 has 37 of data
First non-zero cases is on day 57, and 30 days before 5 days is day 51
84017077 has 66 of data
First non-zero cases is on day 73, and 30 days before 5 days is day 60
84017079 has 57 of data
First non-zero cases is on day 74, and 30 days before 5 days is day 65
84017081 has 52 of data
First non-zero cases is on day 72, and 30 days before 5 days is day 73
84017083 has 44 of data
First non-zero cases is on day 48, and 30 days bef

In [ ]:
stan_data

In [ ]:
# remove travel bans and sport
stan_data["covariate2"] = 0*stan_data["covariate2"]
stan_data["covariate4"] = 0*stan_data["covariate4"]

# self-isolating if ill
stan_data["covariate2"] = stan_data["covariate7"]

In [ ]:
#create the any intervention covariate
stan_data["covariate4"] = 1*(stan_data["covariate1"]+
                            stan_data["covariate3"]+
                            stan_data["covariate4"]+
                            stan_data["covariate5"]+
                            stan_data["covariate6"] > 1)

In [ ]:
stan_data["covariate7"]=0*stan_data["covariate7"]

In [ ]:
import pystan


In [ ]:
sm = pystan.StanModel(file="../../stan-models/base.stan")

In [ ]:
# original
# fit = sm.sampling(data=stan_data,iter=40,warmup=20,chains=2)

# sample as small as possible, just for testing
fit = sm.sampling(data=stan_data,iter=2,warmup=1,chains=1)
print(fit)

In [ ]:
# save it to the file for later use
import pickle
with open('./example.pkl', 'wb') as f:
    pickle.dump(sm, f)

In [ ]:
# import arviz as az
fit.extract()